In [105]:

from libs.datasets import DHBeds
from libs.datasets import JHUDataset
from libs.datasets import CovidTrackingDataSource
from libs.datasets import CDSDataset
from libs.datasets import FIPSPopulation
from libs.datasets.dataset_utils import AggregationLevel
from libs.build_params import US_STATE_ABBREV
from libs.datasets import dataset_utils
from libs.datasets import custom_aggregations
import pandas as pd
pd.set_option('display.max_rows', 10000)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [115]:
data = JHUDataset.local().data
dataset_utils.check_index_values_are_unique(data[data.FIPS.notnull() & (data.FIPS != '99999')], index=['date', 'FIPS'], duplicates_as_error=False)
# timeseries = covid_tracking_source.timeseries()


# data[data.inIcuCurrently.notnull()].head(500)
# len(data[data.current_hospitalized == 0])
# data[['state', 'hospitalizedCumulative', 'hospitalized', 'hospitalizedCurrently']]
# data[['death', 'deathIncrease']]


In [104]:
beds = DHBeds.local().beds().data
beds[beds.state == 'NY'].sort_values('licensed_beds', ascending=False).head(10)

Could not match ('AS', 'American Samoa')
Could not match ('PR', 'Mayaguez')
Could not match ('GU', 'Guam')
Could not match ('MP', 'Saipan')


,fips,state,aggregate_level,country,staffed_beds,licensed_beds,icu_beds,source,generated,max_bed_count,county
2545,NaN,NY,state,USA,53370,49874,3109,DH,True,56175,NaN
2507,36061,NY,county,USA,23233,22095,1328,DH,True,24422,New York County
1605,36059,NY,county,USA,4187,4377,235,DH,False,4377,Nassau County
1629,36119,NY,county,USA,3160,3396,158,DH,False,3396,Westchester County
1623,36103,NY,county,USA,3509,3181,250,DH,False,3509,Suffolk County
1593,36029,NY,county,USA,2653,2664,190,DH,False,2664,Erie County
1608,36067,NY,county,USA,2113,1762,118,DH,False,2113,Onondaga County
1603,36055,NY,county,USA,2038,1139,154,DH,False,2038,Monroe County
1580,36001,NY,county,USA,1623,899,81,DH,False,1623,Albany County
1610,36071,NY,county,USA,986,807,76,DH,False,986,Orange County


In [ ]:
data.index.names

In [ ]:
jhu = JHUDataset.local()
jhu._raw_data[jhu._raw_data.FIPS == '49053'][['date', 'Province_State', 'Admin2', 'FIPS']].set_index('date')
# dataset_utils.check_index_values_are_unique(data).index

# data.loc[('2020-02-01', 'USA', 'MA', '99999'),:]



In [ ]:
timeseries = JHUDataset.local().timeseries()
usa_timeseries = timeseries.get_subset(None, country="USA")
data = usa_timeseries.latest_values(AggregationLevel.COUNTY)
data[data.fips != '99999'].date.unique()

In [ ]:
data = pd.read_csv('/Users/chris/Downloads/patch_location_data.csv', dtype={'county_fips': str})
data = data.rename({'county_fips': 'fips'}, axis=1).set_index('fips')
combined = pop.set_index('fips').join(data, how='outer')

In [ ]:
assert not len(data[data.internal_county_tid.isnull()])
assert not len(pop[pop.country.isnull()])

missing_in_patch = combined[combined.internal_county_tid.isnull()].reset_index()
missing_in_patch = missing_in_patch[missing_in_patch.fips != '99999']
missing_in_ours = combined[combined.country.isnull()].reset_index()


print("missing in patch")
missing_in_patch[['fips', 'state', 'county']]


In [ ]:
print("missing in ours")
missing_in_ours[['fips', 'county_name', 'state_id']]

In [ ]:
jhu_county = jhu.get_subset(None, state='NY').data
print(jhu_county.head())
group = ['date', 'country', 'state', 'aggregate_level', 'source']
jhu_county = custom_aggregations.update_with_combined_new_york_counties(jhu_county, group)
jhu_county[jhu_county.state == 'NY']

In [ ]:
data = []

for state in beds.data.state.unique():
#     print(state)
    try:
        data.append({
            "state": state, 
            "new": beds.get_state_level(state),
            "legacy": old_data.get_beds_by_country_state('USA', state), 
            "delta": old_data.get_beds_by_country_state('USA', state) - beds.get_state_level(state),
        })
    except:
        print("HI")

df = pd.DataFrame(data)


In [ ]:
DHBeds.local().beds()

In [ ]:
from libs.datasets import DHBeds
from libs.datasets import JHUDataset
from libs.datasets import FIPSPopulation
from libs.datasets.dataset_utils import AggregationLevel

population_data = FIPSPopulation.local().population().data
cases_data = JHUDataset.local().timeseries().get_subset(None, country='USA').latest_values(AggregationLevel.COUNTY)
beds_data = DHBeds.local().beds().data

index = ['fips', 'state', 'aggregate_level']
population = population_data.set_index(index)
beds = beds_data.set_index(index)
cases = cases_data.set_index(index)

total = population.join(beds, on=index, rsuffix='_beds')
total = total.join(cases, on=index, rsuffix='_cases').reset_index()
total = total[total.aggregate_level == 'county']
# TODO: Deal with unknowns
total = total[total.fips != '99999']t
total[['fips', 'state', 'county', 'population', 'licensed_beds', 'cases', 'deaths', 'recovered']].to_csv('data_overview_2.csv')



In [ ]:
fips_data = FIPSPopulation.local().data

In [ ]:
beds_data.county_data[beds_data.county_data.state == 'MA'].head(100).to_dict(orient='records')

In [ ]:
from libs.datasets.sources import google_mobility
from libs import build_params

In [ ]:
data = google_mobility.GoogleMobilityReport.local()
data.data.to_csv('google_mobility.csv')

In [ ]:
parsed = []

def _category_list_to_dict(categories):
    data = {}
    for category in categories:
        data[category['category']] = category.get('percent', 0) / 100.0
        
    return data


parsed_data = []
for record in data:
    country_code = record['country_code']
    country, state_long = country_code.split('_')
    if country == 'US':
        country = 'USA'
    state = build_params.US_STATE_ABBREV[state_long]
    categories = _category_list_to_dict(record['national_data'])
    
    state_data = {
        'country': country,
        'state': state,
        'aggregate_level': 'state'
    }
    state_data.update(categories)
    parsed_data.append(state_data)
    
    for county_record in record['county_data']:
        print(county_record)
        county = county_record['county_name']
        county_category = _category_list_to_dict(county_record['data'])
        county_record = {
            'country': country,
            'state': state,
            'county': county,
            'aggregate_level': 'county',
        }
        county_record.update(county_category)
        parsed_data.append(county_record)


In [ ]:
pd.DataFrame(parsed_data)

In [ ]:
jhu = JHUDataset.local()
df = jhu.data[(jhu.data[jhu.Fields.STATE] == 'NM') & (jhu.data[jhu.Fields.AGGREGATE_LEVEL] == 'county')][['date''Admin2', '']]
# df[df.date == '2020-03-31'][['FIPS', 'Province_State', 'Confirmed', 'Admin2', 'Combined_Key']].sort_values('Confirmed', ascending=False)

In [ ]:
def build_fips_data_frame(census_csv, counties_csv):
    counties = pd.read_csv(counties_csv, dtype=str)
    counties.columns = [
        "summary",
        "state_fip",
        "county_fip",
        "subdivision",
        "place",
        "city",
        "name",
    ]

    county_pop = pd.read_csv(census_csv)
    county_pop.columns = ["county_state", "population"]

    # Various filters
    no_county = counties.county_fip == "000"
    has_state = counties.state_fip != "00"
    has_county = counties.county_fip != "000"
    no_subdivision = counties.subdivision == "00000"
    no_place = counties.place == "00000"
    no_city = counties.city == "00000"

    # Create state level fips
    states = counties[
        has_state & no_county & no_subdivision & no_city & no_place
    ].reset_index()
    states = states.rename({"name": "state"}, axis=1)[["state_fip", "state"]]
    states.state = states.state.apply(lambda x: US_STATE_ABBREV[x])

    # Create County level
    county_only = counties[
        has_county & no_subdivision & no_place & no_city
    ].reset_index()
    county_only = county_only.rename({"name": "county"}, axis=1)
    county_only["fips"] = county_only.state_fip + county_only.county_fip
    state_data = (
        county_only.set_index("state_fip")
        .join(states.set_index("state_fip"), on="state_fip")
        .reset_index()
    )

    # Sorry these lambdas are ugly
    county_pop.population = county_pop.population.apply(
        lambda x: int(x.replace(",", ""))
    )
    county_pop["state"] = county_pop.county_state.apply(
        lambda x: US_STATE_ABBREV[x.split(",")[1].strip()]
    )
    county_pop["county"] = county_pop.county_state.apply(
        lambda x: x.split(",")[0].strip().replace(".", "")
    )

    left = state_data.set_index(["state", "county"])
    right = county_pop.set_index(["state", "county"])
    results = left.join(right, on=["state", "county"]).reset_index()
    return results[["state", "county", "fips", "population"]]


In [ ]:
county_pop = pd.read_csv('/Users/chris/Documents/co-est2019-annres.csv')
county_pop.columns = ["county_state", "population"]

county_pop.population = county_pop.population.apply(
    lambda x: int(x.replace(",", ""))
)
county_pop["state"] = county_pop.county_state.apply(
    lambda x: US_STATE_ABBREV[x.split(",")[1].strip()]
)
county_pop["county"] = county_pop.county_state.apply(
    lambda x: x.split(",")[0].strip().lstrip('.')
)

county_pop = county_pop.replace('Sainte', 'Ste.')
county_pop = county_pop.replace('Saint', 'St.')
state_data = pd.read_csv('/Users/chris/personal/covid-data-model/libs/datasets/sources/fips_population.csv', dtype={'fips': str})
left = state_data.drop('population', axis=1).set_index(["state", "county"])
right = county_pop.set_index(["state", "county"])
results = left.join(right, on=["state", "county"]).reset_index()
results[['state', 'county', 'fips', 'population']]  # .to_csv('/Users/chris/personal/covid-data-model/libs/datasets/sources/fips_population.csv')